# Objetivo

Primeira parte de exploração de modelos e avaliação de métricas. Dois tipos de modelos serão priorizados, a RandomForest (bagging) e o XGBoost (boosting). Para estes dois, serão usados os hiperparâmetros default e o tunig de hiperparâmetros via Bayesian Search.

Dado o comportamento assimétrico da variável resposta, não será considerado a regressão linear múltipla nesse caso. Como existem casos de não pagamento do saldo em atraso, não se pode aplicar a transformação logarítmica.

# Pacotes

In [1]:
import pandas as pd
import numpy as np
import mlflow
from mlflow.models import infer_signature
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from bayes_opt import BayesianOptimization
from hyperopt import fmin, tpe, Trials, hp
from functools import partial

In [12]:
pd.set_option('display.max_rows', None)

# Leitura das bases de treino e validação

In [2]:
dados_treino_pp = pd.read_parquet('../0.Avalia_Dados/Dados/treino_pp.parquet')
dados_val_pp = pd.read_parquet('../0.Avalia_Dados/Dados/validacao_pp.parquet')

In [ ]:
# Volumetria das bases de treino e validação

dados_treino_pp.shape, dados_val_pp.shape

((109840, 72), (27460, 75))

In [ ]:
# Colunas da base de treino

dados_treino_pp.columns

Index(['IDADE', 'VAR_4', 'VAR_5', 'VAR_7', 'VAR_8', 'VAR_9', 'VAR_11',
       'VAR_12', 'VAR_15', 'VAR_17', 'VAR_21', 'VAR_22', 'VAR_23', 'VAR_32',
       'VAR_33', 'VAR_35', 'VAR_40', 'VAR_138', 'VAR_139', 'VAR_142',
       'VAR_143', 'VAR_146', 'VAR_147', 'VAR_149', 'VAR_150', 'VAR_151',
       'VAR_157', 'VAR_158', 'VAR_159', 'VAR_160', 'VAR_170', 'VAR_173',
       'VAR_174', 'VAR_176', 'VAR_178', 'VAR_179', 'VAR_180', 'VAR_188',
       'VAR_210', 'VAR_222', 'VAR_232', 'VAR_253', 'VAR_254', 'VAR_264',
       'VAR_265', 'VAR_267', 'VAR_268', 'VAR_269', 'VAR_305', 'VAR_309',
       'VAR_310', 'VAR_313', 'dias_atraso', 'pagamento', 'segmento_veiculo',
       'VAR_2', 'VAR_42', 'VAR_44', 'VAR_45', 'VAR_46', 'VAR_47', 'VAR_48',
       'VAR_50', 'VAR_51', 'VAR_52', 'VAR_53', 'VAR_55', 'VAR_56', 'VAR_57',
       'VAR_113', 'VAR_256', 'saldo_vencido'],
      dtype='object')

In [ ]:
# Colunas das base de validação (existem algumas a mais do que na base de treino)

dados_val_pp.columns

Index(['id', 'documento', 'mes_pagamento', 'IDADE', 'VAR_4', 'VAR_5', 'VAR_7',
       'VAR_8', 'VAR_9', 'VAR_11', 'VAR_12', 'VAR_15', 'VAR_17', 'VAR_21',
       'VAR_22', 'VAR_23', 'VAR_32', 'VAR_33', 'VAR_35', 'VAR_40', 'VAR_138',
       'VAR_139', 'VAR_142', 'VAR_143', 'VAR_146', 'VAR_147', 'VAR_149',
       'VAR_150', 'VAR_151', 'VAR_157', 'VAR_158', 'VAR_159', 'VAR_160',
       'VAR_170', 'VAR_173', 'VAR_174', 'VAR_176', 'VAR_178', 'VAR_179',
       'VAR_180', 'VAR_188', 'VAR_210', 'VAR_222', 'VAR_232', 'VAR_253',
       'VAR_254', 'VAR_264', 'VAR_265', 'VAR_267', 'VAR_268', 'VAR_269',
       'VAR_305', 'VAR_309', 'VAR_310', 'VAR_313', 'dias_atraso',
       'saldo_vencido', 'pagamento', 'segmento_veiculo', 'VAR_2', 'VAR_42',
       'VAR_44', 'VAR_45', 'VAR_46', 'VAR_47', 'VAR_48', 'VAR_50', 'VAR_51',
       'VAR_52', 'VAR_53', 'VAR_55', 'VAR_56', 'VAR_57', 'VAR_113', 'VAR_256'],
      dtype='object')

In [ ]:
# Separação das bases explicativa e resposta

X_treino = dados_treino_pp.drop(columns=['saldo_vencido', 'pagamento'])
y_treino = dados_treino_pp['pagamento']

X_val = dados_val_pp.drop(columns=['saldo_vencido', 'pagamento', 'id', 'documento', 'mes_pagamento'])
y_val = dados_val_pp['pagamento']

In [4]:
X_treino.shape, X_val.shape

((109840, 70), (27460, 70))

# Criação do experimento

In [ ]:
# Cria um experimento no MLflow e as diferentes runs serão associadas a esse experimento: executado somente uma vez

# experiment = mlflow.create_experiment(name = 'Modelos',
#                                      artifact_location = 'Artf_Modelos',
#                                      tags = {'Environment': 'Development', 'Version': '1.0.0'})

In [ ]:
# experiment

'401791776067865015'

In [ ]:
# Depois da criação do experimento anteriormente, basta configurá-lo para que que as informações sejam logadas nele

experiment = mlflow.set_experiment(experiment_id = '401791776067865015')
experiment.experiment_id

'401791776067865015'

# Modelos

## Sem otimização de hiperparâmetros

In [ ]:
# Random Forest sem otimização de hiperparâmetros (default)

if __name__ == '__main__':
    
    with mlflow.start_run(run_name = 'RF_1', experiment_id = experiment.experiment_id) as run: 
        
        # Cria o modelo
        rf = RandomForestRegressor()
        
        # Treina o modelo
        rf.fit(X_treino, y_treino)

        mlflow.log_params(rf.get_params())

        # Log das métricas na base de TREINO
        mlflow.log_metric('MSE_Treino', mean_squared_error(y_treino, rf.predict(X_treino)))
        mlflow.log_metric('RMSE_Treino', root_mean_squared_error(y_treino, rf.predict(X_treino)))
        mlflow.log_metric('MAE_Treino', mean_absolute_error(y_treino, rf.predict(X_treino)))
        mlflow.log_metric('R2_Treino', r2_score(y_treino, rf.predict(X_treino)))

        # Log das métricas na base de VALIDAÇÃO
        mlflow.log_metric('MSE_Val', mean_squared_error(y_val, rf.predict(X_val)))
        mlflow.log_metric('RMSE_Val', root_mean_squared_error(y_val, rf.predict(X_val)))
        mlflow.log_metric('MAE_Val', mean_absolute_error(y_val, rf.predict(X_val)))
        mlflow.log_metric('R2_Val', r2_score(y_val, rf.predict(X_val)))

        signature = infer_signature(X_treino, rf.predict(X_treino))
        mlflow.sklearn.log_model(rf, signature=signature, artifact_path='modelo')

/home/hugo/Documents/Git_GitHub/Estudo_Cobranca/vCobranca/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [ ]:
# XGBoost sem otimização de hiperparâmetros (default)

if __name__ == '__main__':
    
    with mlflow.start_run(run_name = 'XGB_1', experiment_id = experiment.experiment_id) as run: 
        
        # Cria o modelo
        XGB = xgb.XGBRegressor()
        
        # Treina o modelo
        XGB.fit(X_treino, y_treino)

        mlflow.log_params(XGB.get_params())

        # Log das métricas na base de TREINO
        mlflow.log_metric('MSE_Treino', mean_squared_error(y_treino, XGB.predict(X_treino)))
        mlflow.log_metric('RMSE_Treino', root_mean_squared_error(y_treino, XGB.predict(X_treino)))
        mlflow.log_metric('MAE_Treino', mean_absolute_error(y_treino, XGB.predict(X_treino)))
        mlflow.log_metric('R2_Treino', r2_score(y_treino, XGB.predict(X_treino)))

        # Log das métricas na base de VALIDAÇÃO
        mlflow.log_metric('MSE_Val', mean_squared_error(y_val, XGB.predict(X_val)))
        mlflow.log_metric('RMSE_Val', root_mean_squared_error(y_val, XGB.predict(X_val)))
        mlflow.log_metric('MAE_Val', mean_absolute_error(y_val, XGB.predict(X_val)))
        mlflow.log_metric('R2_Val', r2_score(y_val, XGB.predict(X_val)))

        signature = infer_signature(X_treino, XGB.predict(X_treino))
        mlflow.sklearn.log_model(XGB, signature=signature, artifact_path='modelo')

/home/hugo/Documents/Git_GitHub/Estudo_Cobranca/vCobranca/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


## Com otimização dos hiperparâmetros

### Função objetivo

In [5]:
# Função objetivo com validação cruzada
# A otimização será feita considerando o comportamento médio do MSE nos folds de validação

def func_objetivo_CV(parametros, modelo, folds, expr, X, y):
    # função objetivo para "minimizar", mas dependendo da métrica de interesse, na realidade, é maximizar 
    # parametros é o espaço paramétrico a ser explorado
    # expr é uma string que representa o id do experimento que foi criado
    # modelo é uma string de qual modelo será rodado: Random Forest ou XGBoost
    # folds é um int que diz quantos folds de validação serão usados
    # X e y são as bases que serão aplicadas o cross-validation

    # O output é o valor do score a ser minimizado/maximizado
    
    with mlflow.start_run(nested = True, experiment_id=expr) as run:

        if modelo == 'RF':
            reg = RandomForestRegressor(**parametros) 
            reg.fit(X, y)
        elif modelo == 'XGB':
            reg = xgb.XGBRegressor(**parametros)
            reg.fit(X, y)
        
        score = cross_val_score(estimator = reg, X = X, y = y, cv = folds, scoring='neg_mean_squared_error').mean()

        # Log de parâmetros e métricas

        mlflow.log_params(reg.get_params())
        mlflow.log_metric('Neg_MAE_cv', cross_val_score(estimator = reg, X = X, y = y, cv = folds, scoring='neg_mean_absolute_error').mean())
        mlflow.log_metric('Neg_MSE_cv', cross_val_score(estimator = reg, X = X, y = y, cv = folds, scoring='neg_mean_squared_error').mean())
        mlflow.log_metric('Neg_RMSE_cv', cross_val_score(estimator = reg, X = X, y = y, cv = folds, scoring='neg_root_mean_squared_error').mean())
        mlflow.log_metric('R2_cv', cross_val_score(estimator = reg, X = X, y = y, cv = folds, scoring='r2').mean())
        
        signature = infer_signature(X, reg.predict(X))
        mlflow.sklearn.log_model(reg, signature=signature, artifact_path='modelo')

    return -score

In [ ]:
# Otimização para a Random Forest aplicada somente a uma amostra da base de treino (computacionalmente mais barato)

if __name__ == '__main__':
    
    space = {
        "n_estimators": hp.choice('n_estimators', np.arange(10, 500, dtype=int)),
        "max_depth": hp.choice('max_depth', np.arange(10, 300, dtype=int)),
        "min_samples_leaf": hp.choice('min_samples_leaf', np.arange(200, 500, dtype=int)),
        "min_samples_split": hp.choice('min_samples_split', np.arange(200, 500, dtype=int))
    }
    
    with mlflow.start_run(run_name = 'RF_CV', experiment_id=experiment.experiment_id) as run:
        best_params = fmin(
            fn = partial(
                func_objetivo_CV,
                expr = experiment.experiment_id,
                modelo = 'RF',
                X = X_treino[0:1000],
                y = y_treino[0:1000],
                folds = 3
            ),
            space = space,
            algo = tpe.suggest,
            max_evals = 25,
            trials = Trials(),
            timeout = 10
        )
       
        # Identificado o melhor conjunto de hiperparâmetros, treina o modelo com toda a base de treino e metrifica os escores na base de validação

        reg = RandomForestRegressor(**best_params)
        reg.fit(X_treino, y_treino)
                   
        mlflow.log_params(reg.get_params())

        # Log das métricas na base de TREINO
        mlflow.log_metric('MSE_Treino', mean_squared_error(y_treino, reg.predict(X_treino)))
        mlflow.log_metric('RMSE_Treino', root_mean_squared_error(y_treino, reg.predict(X_treino)))
        mlflow.log_metric('MAE_Treino', mean_absolute_error(y_treino, reg.predict(X_treino)))
        mlflow.log_metric('R2_Treino', r2_score(y_treino, reg.predict(X_treino)))

        # Log das métricas na base de VALIDAÇÃO
        mlflow.log_metric('MSE_Val', mean_squared_error(y_val, reg.predict(X_val)))
        mlflow.log_metric('RMSE_Val', root_mean_squared_error(y_val, reg.predict(X_val)))
        mlflow.log_metric('MAE_Val', mean_absolute_error(y_val, reg.predict(X_val)))
        mlflow.log_metric('R2_Val', r2_score(y_val, reg.predict(X_val)))

        signature = infer_signature(X_treino, reg.predict(X_treino))
        mlflow.sklearn.log_model(reg, signature=signature, artifact_path='modelo')

  0%|          | 0/25 [00:00<?, ?trial/s, best loss=?]

/home/hugo/Documents/Git_GitHub/Estudo_Cobranca/vCobranca/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(



  4%|▍         | 1/25 [00:13<05:14, 13.12s/trial, best loss: 206642.78035474592]


/home/hugo/Documents/Git_GitHub/Estudo_Cobranca/vCobranca/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [ ]:
# Otimização para o XGBoost aplicado a toda a base de treino

if __name__ == '__main__':
    
    space = {
        "n_estimators": hp.choice('n_estimators', np.arange(10, 500, dtype=int)),
        "max_depth": hp.choice('max_depth', np.arange(10, 300, dtype=int)),
        "colsample_bytree": hp.quniform('colsample_bytree', 0.2, 1, 0.05),
        "subsample": hp.quniform('subsample', 0.2, 1, 0.05),
        "colsample_bynode": hp.quniform('colsample_bynode', 0.2, 1, 0.05),
        "learning_rate": hp.quniform('learning_rate', 0.0025, 0.5, 0.025)
    }
    
    with mlflow.start_run(run_name = 'XGB_CV', experiment_id=experiment.experiment_id) as run:
        best_params = fmin(
            fn = partial(
                func_objetivo_CV,
                expr = experiment.experiment_id,
                modelo = 'XGB',
                X = X_treino,
                y = y_treino,
                folds = 3
            ),
            space = space,
            algo = tpe.suggest,
            max_evals = 25,
            trials = Trials(),
            timeout = 10
        )
       
        # Identificado o melhor conjunto de hiperparâmetros, treina o modelo com toda a base de treino e metrifica os escores na base de validação

        reg = xgb.XGBRegressor(**best_params)
        reg.fit(X_treino, y_treino)
                   
        mlflow.log_params(reg.get_params())

        # Log das métricas na base de TREINO
        mlflow.log_metric('MSE_Treino', mean_squared_error(y_treino, reg.predict(X_treino)))
        mlflow.log_metric('RMSE_Treino', root_mean_squared_error(y_treino, reg.predict(X_treino)))
        mlflow.log_metric('MAE_Treino', mean_absolute_error(y_treino, reg.predict(X_treino)))
        mlflow.log_metric('R2_Treino', r2_score(y_treino, reg.predict(X_treino)))

        # Log das métricas na base de VALIDAÇÃO
        mlflow.log_metric('MSE_Val', mean_squared_error(y_val, reg.predict(X_val)))
        mlflow.log_metric('RMSE_Val', root_mean_squared_error(y_val, reg.predict(X_val)))
        mlflow.log_metric('MAE_Val', mean_absolute_error(y_val, reg.predict(X_val)))
        mlflow.log_metric('R2_Val', r2_score(y_val, reg.predict(X_val)))

        signature = infer_signature(X_treino, reg.predict(X_treino))
        mlflow.sklearn.log_model(reg, signature=signature, artifact_path='modelo')

  0%|          | 0/25 [00:00<?, ?trial/s, best loss=?]

/home/hugo/Documents/Git_GitHub/Estudo_Cobranca/vCobranca/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(



  4%|▍         | 1/25 [22:57<9:11:03, 1377.64s/trial, best loss: 696514.5101954734]


/home/hugo/Documents/Git_GitHub/Estudo_Cobranca/vCobranca/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [ ]:
# Outra estratégia para a procura do melhor conjunto de hiperparâmetros

# https://drlee.io/step-by-step-guide-bayesian-optimization-with-random-forest-fdc6f329db9c

parametros = {
    'n_estimators': (10, 250),
    'max_depth': (1, 50),
    'min_samples_split': (2, 25),
    'max_features': (0.1, 0.999),
}

In [10]:
def objetivo(n_estimators, max_depth, min_samples_split, max_features):

    model = RandomForestRegressor(n_estimators=int(n_estimators),
                                  max_depth=int(max_depth),
                                  min_samples_split=int(min_samples_split),
                                  max_features=min(max_features, 0.999),  
                                  random_state=42)
    
    return cross_val_score(model, X_treino[0:20000], y_treino[0:20000], cv=3, scoring="neg_mean_squared_error").mean()

In [11]:
optimizer = BayesianOptimization(f=objetivo, pbounds=parametros, random_state=42)
optimizer.maximize(init_points=5, n_iter=10)

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | -1.868e+0 | 19.35     | 0.9547    | 18.84     | 153.7     |
| 2         | -1.866e+0 | 8.645     | 0.2402    | 3.336     | 217.9     |
| 3         | -1.876e+0 | 30.45     | 0.7366    | 2.473     | 242.8     |
| 4         | -1.89e+05 | 41.79     | 0.2909    | 6.182     | 54.02     |
| 5         | -1.877e+0 | 15.91     | 0.5718    | 11.93     | 79.89     |
| 6         | -1.861e+0 | 9.766     | 0.3823    | 7.065     | 201.0     |
| 7         | -1.888e+0 | 50.0      | 0.1       | 2.0       | 185.3     |
| 8         | -1.874e+0 | 10.47     | 0.1492    | 6.317     | 200.4     |
| 9         | -1.86e+05 | 9.162     | 0.4064    | 4.059     | 217.9     |
| 10        | -1.863e+0 | 8.869     | 0.8405    | 4.506     | 218.8     |
| 11        | -1.863e+0 | 9.21      | 0.5192    | 7.785     | 202.8     |
| 12        | -1.89e+05 | 10.42     | 

In [14]:
best_params = optimizer.max['params']
best_params

{'max_depth': np.float64(9.161877360336057),
 'max_features': np.float64(0.40643195882749794),
 'min_samples_split': np.float64(4.058762190729727),
 'n_estimators': np.float64(217.89989410048233)}

In [15]:
modelo_final = RandomForestRegressor(n_estimators=int(best_params['n_estimators']),
                                    max_depth=int(best_params['max_depth']),
                                    min_samples_split=int(best_params['min_samples_split']),
                                    max_features=best_params['max_features'],
                                    random_state=42)
modelo_final.fit(X_treino, y_treino)

RandomForestRegressor(max_depth=9, max_features=np.float64(0.40643195882749794),
                      min_samples_split=4, n_estimators=217, random_state=42)

In [16]:
mean_squared_error(y_treino, modelo_final.predict(X_treino)), root_mean_squared_error(y_treino, modelo_final.predict(X_treino)), mean_absolute_error(y_treino, modelo_final.predict(X_treino)), r2_score(y_treino, modelo_final.predict(X_treino))

(498521.2626987858, 706.060381765459, 499.8298225015349, 0.13795983492655395)

In [17]:
mean_squared_error(y_val, modelo_final.predict(X_val)), root_mean_squared_error(y_val, modelo_final.predict(X_val)), mean_absolute_error(y_val, modelo_final.predict(X_val)), r2_score(y_val, modelo_final.predict(X_val))

(538044.434338212, 733.5151220923888, 507.8611446864369, 0.07301270442652541)

In [18]:
# Registra o modelo no MLflow

if __name__ == '__main__':
    
    with mlflow.start_run(run_name = 'RF_2', experiment_id = experiment.experiment_id) as run: 
        
        mlflow.log_params(modelo_final.get_params())

        # Log das métricas na base de TREINO
        mlflow.log_metric('MSE_Treino', mean_squared_error(y_treino, modelo_final.predict(X_treino)))
        mlflow.log_metric('RMSE_Treino', root_mean_squared_error(y_treino, modelo_final.predict(X_treino)))
        mlflow.log_metric('MAE_Treino', mean_absolute_error(y_treino, modelo_final.predict(X_treino)))
        mlflow.log_metric('R2_Treino', r2_score(y_treino, modelo_final.predict(X_treino)))

        # Log das métricas na base de VALIDAÇÃO
        mlflow.log_metric('MSE_Val', mean_squared_error(y_val, modelo_final.predict(X_val)))
        mlflow.log_metric('RMSE_Val', root_mean_squared_error(y_val, modelo_final.predict(X_val)))
        mlflow.log_metric('MAE_Val', mean_absolute_error(y_val, modelo_final.predict(X_val)))
        mlflow.log_metric('R2_Val', r2_score(y_val, modelo_final.predict(X_val)))

        signature = infer_signature(X_treino, modelo_final.predict(X_treino))
        mlflow.sklearn.log_model(modelo_final, signature=signature, artifact_path='modelo')

/home/hugo/Documents/Git_GitHub/Estudo_Cobranca/vCobranca/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


# Conclusão

- Foram testadas duas famílias de modelos RandomForest e XGBoost, com e sem aplicação de tuning de hiperparâmetros;

- Depois de comparar as métricas na base de validação, optou-se por usar a RandomForest com a primeira opção de search de hiperparâmetros (uso do hyperopt).